In [ ]:
# Import necessary libraries
import nltk
import requests
from bs4 import BeautifulSoup
from googletrans import Translator
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from gtts import gTTS
import pyttsx3
import cv2
import pytesseract
import speech_recognition as sr

# Create ChatBot object and train it with the ChatterBot corpus
chatbot = ChatBot("My ChatBot")
trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train("chatterbot.corpus.english.greetings",
              "chatterbot.corpus.english.conversations")

# Function to process user input using NLP techniques
def process_input(user_input, language):
  # Translate input to English if it is not already in English
  if language != "en":
    translator = Translator()
    user_input = translator.translate(user_input, dest="en").text
  # Tokenize input
  tokens = nltk.word_tokenize(user_input)
  # Perform lemmatization and stemming
  lemmatizer = nltk.WordNetLemmatizer()
  stemmed_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  stemmed_tokens = [nltk.PorterStemmer().stem(token) for token in stemmed_tokens]
  return stemmed_tokens

# Function to process response from ChatBot using NLP techniques and generate voice output
def generate_response(response, language, voice):
  # Translate response to desired language if it is not already in the desired language
  if language != "en":
    translator = Translator()
    response = translator.translate(response, dest=language).text
  # Tokenize response
  tokens = nltk.word_tokenize(response)
  # Perform lemmatization and stemming
  lemmatizer = nltk.WordNetLemmatizer()
  stemmed_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  stemmed_tokens = [nltk.PorterStemmer().stem(token) for token in stemmed_tokens]
  # Generate voice output using gTTS or pyttsx3
  if voice == "gTTS":
    response_voice = gTTS(" ".join(stemmed_tokens))
    response_voice.save("response.mp3")
  elif voice == "pyttsx3":
    engine = pyttsx3.init()
    engine.setProperty("voice", language)
    engine.say(" ".join(stemmed_tokens))
    engine.runAndWait()

# Function to send user input to Google and return search results
def google_search(user_input):
  query = user_input.replace(" ", "+")
  url = "https://www.google.com/search?q=" + query
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
  }
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")
  results = []
  for item in soup.find_all("div", class_="g"):
    link = item.find("a")["href"]
    title = item.find("h3").text
    description = item.find("span", class_="st").text
    results.append({"link": link, "title": title, "description": description})
  return results
